In [1]:
from tqdm import tqdm
import pandapower as pp
import pandas as pd
import numpy as np
from pandapower.pypower.makePTDF import makePTDF
import pandapower.converter as pc
import pyomo.environ as pyo
from scipy.sparse import lil_matrix
from matpowercaseframes import CaseFrames
import pandas as pd
from scipy.stats import multivariate_normal
from numba import njit

In [2]:
%%time
mpc_file = 'julia\\pglib-opf-17.08\\pglib_opf_case30_ieee.m'
mpc_frames = CaseFrames(mpc_file)
mpc = CaseFrames(mpc_file)
net = pc.from_mpc(mpc_file)
ppc = {key: mpc_frames.__getattribute__(key) if not isinstance(
    mpc_frames.__getattribute__(key), pd.DataFrame) else mpc_frames.__getattribute__(
    key).values for key in mpc_frames._attributes}
baseMVA, bus, branch, gen = ppc['baseMVA'], ppc['bus'], ppc['branch'], ppc['gen']
bus_index_dict = {}
for i, index in enumerate(ppc['bus'][:,0]):
    bus_index_dict[index.astype(int)] = i
baseMVA,bus,branch = ppc['baseMVA'],ppc['bus'],ppc['branch']
for i,bus_name in enumerate(bus[:,0]):
    bus[i,0] = bus_index_dict[bus_name]
for i,fbus_name in enumerate(branch[:,0]):
    branch[i,0] = bus_index_dict[fbus_name]
for i,tbus_name in enumerate(branch[:,1]):
    branch[i,1] = bus_index_dict[tbus_name]
for i,gen_name in enumerate(gen[:,0]):
    gen[i,0] = bus_index_dict[gen_name]    
gen_index = gen[:,0].astype(int)
bus_index = bus[:,0].astype(int)
gen_lb = mpc_frames.gen.PMIN.values
gen_ub = mpc_frames.gen.PMAX.values
num_samples = 1
nbus = len(mpc_frames.bus)
nline = len(mpc_frames.branch)
cost_coeff = mpc_frames.gencost.loc[:,'COST_2':'COST_0'].values
H = lil_matrix((len(bus_index), len(gen_index)), dtype=int)
for col, gen_bus in enumerate(gen_index):
    H[gen_bus, col] = 1
H = H.tocsr()               
M = makePTDF(baseMVA, bus, branch)
f_max = np.array([rateA for i in range(len(branch)) for j,rateA in enumerate(branch[i]) if j == 5])
p_init = mpc_frames.gen.PG.values
d = mpc_frames.bus.PD.values
sigma_scaling = 0.03
nonzeroindices = np.nonzero(d)[0]
d_nonzeros = d[d!=0]
sigma = (sigma_scaling * d_nonzeros)
mean = np.zeros(len(d_nonzeros))
covariance_matrix = np.diag(sigma)**2
omega_dist = multivariate_normal(mean=mean, cov=covariance_matrix)
omega_samples = omega_dist.rvs(size=num_samples)
omega = np.zeros((num_samples,nbus))
omega[:,nonzeroindices] = omega_samples

CPU times: total: 375 ms
Wall time: 376 ms


In [3]:
%%time
# paper model
model = pyo.ConcreteModel()
# sets
ngen = len(gen_index)
model.ng = pyo.RangeSet(0, ngen-1)
nbus = len(bus_index)
model.nb = pyo.RangeSet(0, nbus-1)
nline = len(ppc['branch'])
model.nl = pyo.RangeSet(0, nline-1)
# variables
model.p = pyo.Var(model.ng, domain=pyo.Reals, initialize=p_init)
model.omega = pyo.Param(model.nb, domain=pyo.Reals, initialize=omega[0] , mutable=True)
# objective (1a)
model.obj = pyo.Objective(expr =
                    sum([cost_coeff[:,0][g]*model.p[g] for g in model.ng])\
            + sum([cost_coeff[:,1][g]*model.p[g] for g in model.ng]) + cost_coeff[:,2].sum()
                    , sense = pyo.minimize
)
# constraint (1b) μ is omitted
model.c1b = pyo.Constraint(expr =
                          sum([model.p[g] for g in model.ng ]) == 
                           sum([d[b] for b in model.nb]) 
                           - sum([model.omega[b] for b in model.nb])
                          )
# constraint (1c)
def c1c_lb_rule(model, g):
    return gen_lb[g] <= model.p[g]
def c1c_ub_rule(model, g):
    return gen_ub[g] >= model.p[g]
model.c1c_lb = pyo.Constraint(model.ng, rule=c1c_lb_rule)
model.c1c_ub = pyo.Constraint(model.ng, rule=c1c_ub_rule)
# constraint (1d) μ is omitted
flow_expr = M@(H.toarray()@model.p + model.omega - (d))
def c1d_lb_rule(model, l):
    return -f_max[l] <= flow_expr[l]
def c1d_ub_rule(model, l):
    return f_max[l] >= flow_expr[l]
model.c1d_lb = pyo.Constraint(model.nl, rule=c1d_lb_rule)
model.c1d_ub = pyo.Constraint(model.nl, rule=c1d_ub_rule)

CPU times: total: 31.2 ms
Wall time: 24.7 ms


In [4]:
# %%time
solver = pyo.SolverFactory('mosek')
infeasible=0
bases_dict = {}
bases_set = set()
for i, value in tqdm(enumerate(omega)):
    for j in model.nb:
        model.omega[j] = value[j]
    res = solver.solve(model)
    if not res.solver.termination_condition == 'optimal':
        infeasible+=1
#     c_list = []
#     for c in model.component_data_objects(pyo.Constraint):
#         if np.isclose(c.slack(), 0, atol=1e-7):
#             c_list.append(c.name)
#     bases_set.add(tuple(c_list))
#     bases_dict[i] = len(bases_set)

1it [00:00,  4.43it/s]


In [8]:
mosek

<module 'mosek' from 'C:\\Users\\firda\\anaconda3\\Lib\\site-packages\\mosek\\__init__.py'>

In [ ]:
pyo.value(model.obj)